In [1]:
! pip uninstall -y torch torchvision torchaudio


In [2]:
! pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp39-cp39-win_amd64.whl (2619.2 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp39-cp39-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.0.2%2Bcu118-cp39-cp39-win_amd64.whl (2.5 MB)
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
      --------------------------------------- 0.1/5.7 MB 3.3 MB/s eta 0:00:02
     -- ------------------------------------- 0.4/5.7 MB 5.1 MB/s eta 0:00:02
     ------ --------------------------------- 1.0/5.7 MB 7.9 MB/s eta 0:00:01
     -------- ------------------------------- 1.2/5.7 MB 9.4 MB/s eta 0:00:01
     --------------- ------------------------ 2.2/5.7 MB 9.8 MB/s eta 0:00:01
     -------------------- ------------------- 3.0/5.7 MB 11.9 MB/s eta 0:00:01
     ------------------------ --------------- 3.5/5.7 MB 12.5 MB/

In [4]:
import os
import torch
import torchvision
torchvision.disable_beta_transforms_warning()
import torchvision.transforms.v2 as T
import matplotlib.pyplot as plt
plt.ion
import numpy as np
import PIL


model = torchvision.models.detection.ssd300_vgg16()
model.load_state_dict(torch.load('C:/Users/zack/Documents/GitHub/SSD_VGG_PyTorch/ssd300_vgg16_gradientAccumulation_noHen.pth'))

In [ ]:
label_dict = {4.0: 'MALL', 1.0: 'AMCO', 3.0: 'GWTE', 6.0: 'NSHO', 2.0: 'GADW', 8.0: 'RNDU', 5.0: 'NOPI', 7.0: 'REDH'}

# distinct colors 
distinct_colors = ['#f032e6', '#ffffff', '#ffe119', '#3cb44b', '#42d4f4',
                    '#f58231', '#e6194B', '#dcbeff', '#469990', '#4363d8']

# label color map for plotting color-coded boxes by class
label_color_map = {k: distinct_colors[i] for i, k in enumerate(label_dict.keys())}

# function for reshaping boxes 
def get_box(boxes):
    boxes = np.array(boxes)
    boxes = boxes.astype('float').reshape(-1, 4)
    if boxes.shape[0] == 1 : return boxes
    return np.squeeze(boxes)


# function for plotting image
def img_show(image, ax = None, figsize = (6, 9)):
    if ax is None:
        fig, ax = plt.subplots(figsize = figsize)
    ax.xaxis.tick_top()
    ax.imshow(image)
    return ax

def plot_bbox_predicted(ax, boxes, labels, scores): # modify plot_bbox to add confidence scores
    # add box to the image and use label_color_map to color-code by bounding box class if exists else 'black'
    ax.add_patch(plt.Rectangle((boxes[:, 0], boxes[:, 1]), boxes[:, 2] - boxes[:, 0], boxes[:, 3] - boxes[:, 1],
                    fill = False,
                    color = label_color_map[labels.item()] if labels.item() in label_color_map else 'black', 
                    linewidth = 1.5))
    
    # add label and score to the bounding box. concatenate label and score to one string. 
    # use label_dict to replace class numbers with class names
    ax.text(boxes[:, 0], boxes[:, 1] - 100,
        s = f"{label_dict[labels.item()]} {scores.item():.2f}",
        color = 'black',
        fontsize = 6,
        verticalalignment = 'top',
        bbox = {'color': label_color_map[labels.item()] if labels.item() in label_color_map else 'black', 'pad': 0})
    return ax


# function for plotting all predictions on images
def plot_predictions(image, boxes, labels, scores, ax = None):
    ax = img_show(image, ax = ax)
    for i in range(len(boxes)):
        box = get_box(boxes[i])
        plot_bbox_predicted(ax, box, labels[i], scores[i])

In [ ]:
dir = 'E:/Zack/Research/Object Detection/Duck_Drone_Detection_Data/filtered_images/'

In [ ]:
def run_model(model, dir):
    # set model to evaluation
    model.eval()
    # get image
    image_files = os.listdir(dir)
    for i in range(8):
        image = PIL.Image.open(os.path.join(dir, image_files[i]))
        
        width, height = image.size

        # convert image to tensor
        image_tensor = T.ToImageTensor()(image)

        # # add batch dimension
        image_tensor = image_tensor.unsqueeze(0)

        image_tensor = image_tensor.to(torch.float32)

        # resize to 300x300
        image_tensor = T.Resize((300, 300), antialias=True)(image_tensor)

        # normalize image
        image_tensor = T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])(image_tensor)
        
        # run model
        with torch.no_grad():
            output = model(image_tensor)

        # get boxes, labels, and scores
        boxes = output[0]['boxes']
        labels = output[0]['labels']
        scores = output[0]['scores']

        # filter out boxes with scores less than 0.5
        boxes = boxes[scores > 0.5]
        labels = labels[scores > 0.5]
        scores = scores[scores > 0.5]

        # rescale boxes to original image size
        boxes[:, 0] *= width / 300
        boxes[:, 1] *= height / 300
        boxes[:, 2] *= width / 300
        boxes[:, 3] *= height / 300

        # plot predictions
        plot_predictions(image, boxes, labels, scores)
        plt.show()

In [ ]:
run_model(model, dir)

In [ ]:
from torch import package

path = "model.pt"
package_name = "DuckNet"
resource_name = "model.pkl"

with package.PackageExporter(path, debug=True) as exp:
    exp.intern("torchvision.**")
    exp.extern("pycocotools.**")
    exp.extern("PIL.**")
    exp.extern("io")
    exp.extern("matplotlib.**")
    exp.extern("sys")
    exp.extern("requests")
    exp.extern("numpy")
    exp.extern("scipy.**")
    exp.extern("h5py")
    exp.extern("__future__")
    exp.save_pickle(package_name, resource_name, model)

In [ ]:
imp = package.PackageImporter(path)
loaded_model = imp.load_pickle(package_name, resource_name)

In [ ]:
run_model(loaded_model, dir)